In [1]:
# Importar Librerías
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import keras_tuner as kt


In [2]:
#Cargar Archivo
df1 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cordoba1")
df2 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira2")
df3 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira3")
df4 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Guajira4")
df5 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia5")
df6 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Antioquia6")
df7 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico7")
df8 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Atlantico8")
df9 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena9")
df10 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Magdalena10")
df11 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar11")
df12 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Cesar12")
df13 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar13")
df14 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Bolivar14")
df15 = pd.read_excel("Datos sin irrigacion.xlsx", sheet_name = "Resumen_Choco15")

# Lista Df
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15]

# Unirlos en uno solo
df = pd.concat(dfs, ignore_index=True)

In [3]:
#Explorar Archivo
df.head()

,Hoja,Station Number,Año,Día,Irradiacion,Min Temp,Max Temp,Temp Promedio,Vapor,Wind Speed,Precipitacion,Departamento,Irrigacion,Produccion,Toneladas por hectaria
0,2000,1,2000,183.5,4593.10,23.34,32.93,28.14,38.22,1.05,5.83,1,1,5969.732058,59.697321
1,2001,1,2001,183.0,4591.66,23.84,33.27,28.56,39.24,1.09,5.84,1,1,5625.796938,56.257969
2,2002,1,2002,183.0,4591.66,24.13,33.74,28.94,40.13,0.94,5.84,1,1,4309.850302,43.098503
3,2003,1,2003,183.0,4591.66,24.08,33.28,28.68,39.50,1.11,5.84,1,1,4121.129416,41.211294
4,2004,1,2004,183.5,4593.10,24.40,33.39,28.90,40.11,1.05,5.83,1,1,4565.883797,45.658838


In [5]:
#Dividir el dataset en 80% entrenamiento y 20% prueba
train = df.sample(frac=0.8, random_state=100)
test = df.drop(train.index)
test.head()

,Irradiacion,Max Temp,Precipitacion,Vapor,Min Temp,Departamento,Toneladas por hectaria
0,4593.10,32.93,5.83,38.22,23.34,1,59.697321
2,4591.66,33.74,5.84,40.13,24.13,1,43.098503
4,4593.10,33.39,5.83,40.11,24.40,1,45.658838
13,4591.66,33.49,5.84,40.40,24.61,1,45.378550
14,4591.66,33.87,5.84,40.02,23.99,1,47.003314


In [6]:
#Separar características (X) y variable objetivo (y)
train_X = train.copy()
test_X = test.copy()
# Asegurar que las características sean de tipo float para evitar errores
train_X = train_X.astype(float)
test_X = test_X.astype(float)

In [7]:
## Extraer columna objetivo
train_y = train_X.pop('Toneladas por hectaria')
test_y = test_X.pop('Toneladas por hectaria')

In [8]:
#Construir modelo
#Capa de normalizacion
norm = tf.keras.layers.Normalization()
# Adaptar la capa de normalización a los datos de entrenamiento
norm.adapt(np.array(train_X))

MODELOS

In [9]:
#Modelos Capas
def construir_modelo(norm_layer, capas, input_shape):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation='relu', input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation='relu'))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo(nombre_modelo, capas, resultados_list):
    modelo = construir_modelo(norm, capas, train_X.shape[1])
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': nombre_modelo,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo  # Guardamos el objeto modelo también
    })

# Lista para resultados
resultados_modelos = []

# Probar modelos con diferentes configuraciones de capas
configuraciones = {
    'Modelo1': [64],
    'Modelo2': [128, 64],
    'Modelo3': [128, 64, 32],
    'Modelo4': [256, 128, 64, 32]
}

for nombre, capas in configuraciones.items():
    entrenar_evaluar_modelo(nombre, capas, resultados_modelos)

# Crear DataFrame
resultados_df = pd.DataFrame(resultados_modelos).drop(columns=['Modelo_Objeto'])
print(resultados_df)

# Seleccionar el mejor modelo por MAE (puedes cambiar a 'RMSE' o 'R2' si prefieres)
mejor_resultado = min(resultados_modelos, key=lambda x: x['MAE'])
print(f"\nMejor modelo: {mejor_resultado['Modelo']} con MAE = {mejor_resultado['MAE']:.4f}")

# Acceder al objeto del mejor modelo
mejor_modelo = mejor_resultado['Modelo_Objeto']

Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 30.5349 - mae: 30.5349 - val_loss: 29.9358 - val_mae: 29.9358
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.1353 - mae: 29.1353 - val_loss: 30.0973 - val_mae: 30.0973
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.2969 - mae: 29.2969 - val_loss: 30.2877 - val_mae: 30.2877
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.3831 - mae: 30.3831 - val_loss: 30.5710 - val_mae: 30.5710
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 32.3445 - mae: 32.3445 - val_loss: 30.9490 - val_mae: 30.9490
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 29.2212 - mae: 29.2212 - val_loss: 31.3507 - val_mae: 31.3507
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.6183 - mae: 29.6183 - val_loss: 31.9993 - val_mae: 31.9993
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 28.7731 - mae: 28.7731 - val_loss: 32.3595 - val_mae: 32.3595
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 30.2316 - mae: 30.2316 - val_loss: 30.4393 - val_mae: 30.4393
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.8067 - mae: 29.8067 - val_loss: 31.7021 - val_mae: 31.7021
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 25.1021 - mae: 25.1021 - val_loss: 32.4075 - val_mae: 32.4075
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 28.3260 - mae: 28.3260 - val_loss: 33.6925 - val_mae: 33.6925
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.9771 - mae: 28.9771 - val_loss: 33.6876 - val_mae: 33.6876
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 27.0697 - mae: 27.0697 - val_loss: 36.0662 - val_mae: 36.0662
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26.2571 - mae: 26.2571 - val_loss: 35.4721 - val_mae: 35.4721
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.9750 - mae: 27.9750 - val_loss: 36.7889 - val_mae: 36.7889
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 30.7337 - mae: 30.7337 - val_loss: 30.3651 - val_mae: 30.3651
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.9943 - mae: 30.9943 - val_loss: 33.6934 - val_mae: 33.6934
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.0635 - mae: 29.0635 - val_loss: 34.0566 - val_mae: 34.0566
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 27.6951 - mae: 27.6951 - val_loss: 35.8008 - val_mae: 35.8008
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.0580 - mae: 27.0580 - val_loss: 36.6074 - val_mae: 36.6074
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.8497 - mae: 27.8497 - val_loss: 34.0144 - val_mae: 34.0144
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.7722 - mae: 28.7722 - val_loss: 34.8731 - val_mae: 34.8731
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 29.6618 - mae: 29.6618 - val_loss: 37.2927 - val_mae: 37.2927
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 31.6114 - mae: 31.6114 - val_loss: 30.6454 - val_mae: 30.6454
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.0211 - mae: 31.0211 - val_loss: 31.5631 - val_mae: 31.5631
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.8660 - mae: 30.8660 - val_loss: 32.4604 - val_mae: 32.4604
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.2422 - mae: 29.2422 - val_loss: 34.0515 - val_mae: 34.0515
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.6350 - mae: 29.6350 - val_loss: 34.8829 - val_mae: 34.8829
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.3606 - mae: 30.3606 - val_loss: 34.2285 - val_mae: 34.2285
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26.5748 - mae: 26.5748 - val_loss: 34.9087 - val_mae: 34.9087
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 25.3801 - mae: 25.3801 - val_loss: 38.4298 - val_mae: 38.4298
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s

In [10]:
#Modelos Activación
# Obtener la mejor arquitectura de capas del punto anterior
mejores_capas = configuraciones[mejor_resultado['Modelo']]

# Lista de funciones de activación a probar
activaciones = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']

# Lista para nuevos resultados por activación
resultados_activaciones = []

def construir_modelo_activacion(norm_layer, capas, input_shape, activacion):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion, input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo_activacion(nombre_modelo, capas, activacion, resultados_list):
    modelo = construir_modelo_activacion(norm, capas, train_X.shape[1], activacion)
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': f"{nombre_modelo}_{activacion}",
        'Activacion': activacion,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo
    })

# Probar activaciones con las mejores capas
for activacion in activaciones:
    entrenar_evaluar_modelo_activacion("MejorModelo", mejores_capas, activacion, resultados_activaciones)

# Crear DataFrame
resultados_activaciones_df = pd.DataFrame(resultados_activaciones).drop(columns=['Modelo_Objeto'])
print(resultados_activaciones_df)

# Elegir el mejor modelo por activación
mejor_activacion_resultado = min(resultados_activaciones, key=lambda x: x['MAE'])
print(f"\nMejor modelo por activación: {mejor_activacion_resultado['Modelo']} con MAE = {mejor_activacion_resultado['MAE']:.4f}")

# Modelo final
modelo_final = mejor_activacion_resultado['Modelo_Objeto']

#Modelos
# Acceder al objeto del mejor modelo
mejor_modelo = mejor_resultado['Modelo_Objeto']
mejores=[mejor_activacion_resultado['Modelo_Objeto'],mejor_resultado['Modelo_Objeto']]


Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 30.0594 - mae: 30.0594 - val_loss: 29.9844 - val_mae: 29.9844
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.8335 - mae: 30.8335 - val_loss: 30.1719 - val_mae: 30.1719
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.5880 - mae: 29.5880 - val_loss: 30.3429 - val_mae: 30.3429
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 31.0757 - mae: 31.0757 - val_loss: 30.6655 - val_mae: 30.6655
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.5236 - mae: 30.5236 - val_loss: 31.0926 - val_mae: 31.0926
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.1294 - mae: 28.1294 - val_loss: 31.5835 - val_mae: 31.5835
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 29.8704 - mae: 29.8704 - val_loss: 32.4799 - val_mae: 32.4799
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 28.1721 - mae: 28.1721 - val_loss: 32.9630 - val_mae: 32.9630
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 30.6452 - mae: 30.6452 - val_loss: 29.8289 - val_mae: 29.8289
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.3628 - mae: 30.3628 - val_loss: 30.0378 - val_mae: 30.0378
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.1585 - mae: 30.1585 - val_loss: 30.1964 - val_mae: 30.1964
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 31.7323 - mae: 31.7323 - val_loss: 30.3407 - val_mae: 30.3407
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.8718 - mae: 30.8718 - val_loss: 30.6767 - val_mae: 30.6767
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.5528 - mae: 27.5528 - val_loss: 30.8707 - val_mae: 30.8707
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.9556 - mae: 28.9556 - val_loss: 31.1190 - val_mae: 31.1190
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 29.4312 - mae: 29.4312 - val_loss: 31.5591 - val_mae: 31.5591
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 29.6601 - mae: 29.6601 - val_loss: 29.9709 - val_mae: 29.9709
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.9188 - mae: 28.9188 - val_loss: 30.0864 - val_mae: 30.0864
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.9228 - mae: 30.9228 - val_loss: 30.2118 - val_mae: 30.2118
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.5017 - mae: 29.5017 - val_loss: 30.3019 - val_mae: 30.3019
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.1325 - mae: 30.1325 - val_loss: 30.4055 - val_mae: 30.4055
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.1901 - mae: 30.1901 - val_loss: 30.5954 - val_mae: 30.5954
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.7162 - mae: 31.7162 - val_loss: 30.8335 - val_mae: 30.8335
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.6350 - mae: 30.6350 - val_loss: 30.8790 - val_mae: 30.8790
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 27.4584 - mae: 27.4584 - val_loss: 29.9111 - val_mae: 29.9111
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 30.6956 - mae: 30.6956 - val_loss: 30.2156 - val_mae: 30.2156
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 27.9864 - mae: 27.9864 - val_loss: 30.3863 - val_mae: 30.3863
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 28.7753 - mae: 28.7753 - val_loss: 30.6391 - val_mae: 30.6391
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.3923 - mae: 29.3923 - val_loss: 31.1283 - val_mae: 31.1283
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 28.0401 - mae: 28.0401 - val_loss: 31.6503 - val_mae: 31.6503
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.6850 - mae: 28.6850 - val_loss: 32.0790 - val_mae: 32.0790
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 31.4338 - mae: 31.4338 - val_loss: 32.6971 - val_mae: 32.6971
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 30.1953 - mae: 30.1953 - val_loss: 29.9862 - val_mae: 29.9862
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 32.0758 - mae: 32.0758 - val_loss: 30.2392 - val_mae: 30.2392
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 29.4397 - mae: 29.4397 - val_loss: 30.3821 - val_mae: 30.3821
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 31.4502 - mae: 31.4502 - val_loss: 30.7164 - val_mae: 30.7164
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.9694 - mae: 28.9694 - val_loss: 30.9051 - val_mae: 30.9051
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.6273 - mae: 29.6273 - val_loss: 31.4749 - val_mae: 31.4749
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 30.3421 - mae: 30.3421 - val_loss: 32.0604 - val_mae: 32.0604
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.9135 - mae: 27.9135 - val_loss: 32.7642 - val_mae: 32.7642
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9

In [11]:
# --- FUNCIONES ---
def construir_modelo(norm_layer, capas, input_shape, activacion='relu'):
    modelo = tf.keras.Sequential([norm_layer])
    for i, neuronas in enumerate(capas):
        if i == 0:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion, input_shape=(input_shape,)))
        else:
            modelo.add(tf.keras.layers.Dense(neuronas, activation=activacion))
    modelo.add(tf.keras.layers.Dense(1))  # Capa de salida
    return modelo

def entrenar_evaluar_modelo(nombre_modelo, capas, resultados_list, activacion='relu'):
    modelo = construir_modelo(norm, capas, train_X.shape[1], activacion)
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='mean_absolute_error',
        metrics=['mae']
    )
    history = modelo.fit(
        train_X,
        train_y,
        epochs=100,
        validation_split=0.2,
        verbose=1
    )
    y_pred = modelo.predict(test_X).flatten()
    mse = mean_squared_error(test_y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(test_y, y_pred)
    r2 = r2_score(test_y, y_pred)

    resultados_list.append({
        'Modelo': nombre_modelo,
        'Capas': str(capas),
        'Activacion': activacion,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Modelo_Objeto': modelo
    })

# --- PRUEBA CON DIFERENTES CAPAS ---
resultados_modelos = []

configuraciones = {
    'Modelo1': [64],
    'Modelo2': [128, 64],
    'Modelo3': [128, 64, 32],
    'Modelo4': [256, 128, 64, 32]
}

for nombre, capas in configuraciones.items():
    entrenar_evaluar_modelo(nombre, capas, resultados_modelos)

# --- MEJOR MODELO SEGÚN MAE ---
mejor_resultado = min(resultados_modelos, key=lambda x: x['MAE'])
mejores_capas = configuraciones[mejor_resultado['Modelo']]

# --- PRUEBA DE ACTIVACIONES CON LAS MEJORES CAPAS ---
activaciones = ['relu', 'tanh', 'sigmoid', 'elu', 'selu']

for activacion in activaciones:
    entrenar_evaluar_modelo(f"MejorModelo_{activacion}", mejores_capas, resultados_modelos, activacion)

# --- CREAR DATAFRAME FINAL ---
resultados_df = pd.DataFrame(resultados_modelos).drop(columns=['Modelo_Objeto'])
print(resultados_df)

# --- ELEGIR EL MEJOR MODELO GLOBAL ---
mejor_modelo_general = min(resultados_modelos, key=lambda x: x['MAE'])
print(f"\n🏆 Mejor modelo total: {mejor_modelo_general['Modelo']} | MAE = {mejor_modelo_general['MAE']:.4f} | Activación: {mejor_modelo_general['Activacion']}")

# --- ACCEDER A LOS OBJETOS DE LOS MEJORES MODELOS ---
modelo_mejor_capas = mejor_resultado['Modelo_Objeto']
modelo_mejor_activacion = mejor_modelo_general['Modelo_Objeto']
mejores = [modelo_mejor_activacion, modelo_mejor_capas]


Epoch 1/100


c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 29.9646 - mae: 29.9646 - val_loss: 29.9234 - val_mae: 29.9234
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.9727 - mae: 30.9727 - val_loss: 30.0253 - val_mae: 30.0253
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 31.4188 - mae: 31.4188 - val_loss: 30.1477 - val_mae: 30.1477
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.2742 - mae: 30.2742 - val_loss: 30.3673 - val_mae: 30.3673
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.6950 - mae: 30.6950 - val_loss: 30.6179 - val_mae: 30.6179
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.4059 - mae: 30.4059 - val_loss: 30.9619 - val_mae: 30.9619
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30.7009 - mae: 30.7009 - val_loss: 31.5394 - val_mae: 31.5394
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.3997 - mae: 29.3997 - val_loss: 31.9927 - val_mae: 31.9927
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 29.8911 - mae: 29.8911 - val_loss: 30.1038 - val_mae: 30.1038
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 32.3278 - mae: 32.3278 - val_loss: 31.4695 - val_mae: 31.4695
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.3348 - mae: 29.3348 - val_loss: 32.9994 - val_mae: 32.9994
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 29.0122 - mae: 29.0122 - val_loss: 32.8521 - val_mae: 32.8521
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.8850 - mae: 28.8850 - val_loss: 34.1998 - val_mae: 34.1998
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 27.5399 - mae: 27.5399 - val_loss: 34.6321 - val_mae: 34.6321
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 27.4438 - mae: 27.4438 - val_loss: 35.5716 - val_mae: 35.5716
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.5472 - mae: 29.5472 - val_loss: 36.3062 - val_mae: 36.3062
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 31.5734 - mae: 31.5734 - val_loss: 29.9946 - val_mae: 29.9946
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.8995 - mae: 29.8995 - val_loss: 30.1386 - val_mae: 30.1386
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.7173 - mae: 30.7173 - val_loss: 30.2700 - val_mae: 30.2700
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.9004 - mae: 30.9004 - val_loss: 30.3668 - val_mae: 30.3668
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.3487 - mae: 30.3487 - val_loss: 30.5251 - val_mae: 30.5251
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.5880 - mae: 30.5880 - val_loss: 30.6737 - val_mae: 30.6737
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.2501 - mae: 31.2501 - val_loss: 30.9085 - val_mae: 30.9085
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.6670 - mae: 30.6670 - val_loss: 31.1268 - val_mae: 31.1268
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 31.3637 - mae: 31.3637 - val_loss: 29.8827 - val_mae: 29.8827
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.7844 - mae: 28.7844 - val_loss: 30.0897 - val_mae: 30.0897
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 31.0306 - mae: 31.0306 - val_loss: 30.2275 - val_mae: 30.2275
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 30.3975 - mae: 30.3975 - val_loss: 30.4838 - val_mae: 30.4838
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 28.3859 - mae: 28.3859 - val_loss: 30.7706 - val_mae: 30.7706
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 27.3756 - mae: 27.3756 - val_loss: 31.1646 - val_mae: 31.1646
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.8142 - mae: 31.8142 - val_loss: 31.9048 - val_mae: 31.9048
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.4011 - mae: 29.4011 - val_loss: 32.1618 - val_mae: 32.1618
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━

c:\Users\anton\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 29.3097 - mae: 29.3097 - val_loss: 29.9033 - val_mae: 29.9033
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.2433 - mae: 28.2433 - val_loss: 30.2132 - val_mae: 30.2132
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 30.1642 - mae: 30.1642 - val_loss: 30.3070 - val_mae: 30.3070
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 30.0782 - mae: 30.0782 - val_loss: 30.5624 - val_mae: 30.5624
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.5518 - mae: 29.5518 - val_loss: 30.9911 - val_mae: 30.9911
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 27.6643 - mae: 27.6643 - val_loss: 31.5103 - val_mae: 31.5103
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.8284 - mae: 29.8284 - val_loss: 32.0858 - val_mae: 32.0858
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 29.0813 - mae: 29.0813 - val_loss: 32.8336 - val_mae: 32.8336
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s